In [1]:
import pandas as pd

df = pd.read_csv("../../../complaints-2020-01-22_08_24.csv", encoding="utf-8")

df_selected = df.loc[:, ('Product', 'Consumer complaint narrative', 'Issue')]

In [2]:
print(df_selected.shape)
df_selected.head()

(485701, 3)


,Product,Consumer complaint narrative,Issue
0,"Credit reporting, credit repair services, or o...",I have complained many times that the credit r...,Credit monitoring or identity theft protection...
1,Debt collection,please review the current fraud account and al...,False statements or representation
2,Debt collection,Called multiple times over the years for a deb...,Attempts to collect debt not owed
3,Debt collection,I sent in a letter to the company to have them...,Attempts to collect debt not owed
4,"Credit reporting, credit repair services, or o...",On XX/XX/19 I applied for a Debt Relief Produc...,Improper use of your report


In case of nulls (should be solved now):

In [3]:
df_selected = df_selected.dropna()

---

__Functions to clean the Consumer complaint narrative__

In [4]:
import re
import string

def clean_url(complaint):
    # to do: more regex url garbage matching
    complaint = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', complaint)
    complaint = re.sub('https? ?: ?// ?(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', complaint)
    return complaint

# Remove punctuation from complaint
def clean_punctuation(complaint):
    complaint = re.sub('[%s]' % re.escape(string.punctuation), '', complaint)
    return complaint

# Remove non-sensical characters from complaint
def clean_nonsense(complaint):
    complaint = re.sub('[''""...]', '', complaint)
    complaint = re.sub('\n', '', complaint)
    return complaint

# Remove censored words from complaint
def clean_censored(complaint):
    complaint = re.sub('[XXXX]', '', complaint)
    return complaint

# Turn everything into lowercase
def clean_lowercase(complaint):
    complaint = complaint.lower()
    return complaint

# Remove numbers from complaint
def clean_numbers(complaint):
    complaint = re.sub('\w*\d\w', '', complaint)
    return complaint

__Application of narrative cleaning__

In [5]:
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_url)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_punctuation)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_nonsense)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_censored)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_lowercase)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_numbers)

**Apply CountVectorizer**

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

**Apply TF-IDF**

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

# Normalise with tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Shape', X_train_tfidf.shape)
print(X_train_tfidf)

Shape (485701, 77541)
  (0, 77176)	0.057603117293145815
  (0, 76652)	0.16256155704199224
  (0, 75309)	0.0704998089772363
  (0, 75075)	0.06367724939199933
  (0, 74968)	0.09219266901371895
  (0, 73727)	0.12517299128438755
  (0, 71967)	0.17949182587880833
  (0, 69841)	0.11897824400517687
  (0, 69713)	0.05790316684384142
  (0, 69322)	0.11526692039409442
  (0, 69116)	0.0919644454692771
  (0, 67077)	0.0945540081983084
  (0, 66813)	0.13729499089431052
  (0, 65093)	0.10163692275565259
  (0, 63443)	0.12076334352602229
  (0, 62631)	0.062129528417944724
  (0, 62615)	0.12209465027976771
  (0, 62146)	0.11073198116064686
  (0, 61559)	0.07623732630893473
  (0, 59674)	0.09011779388563108
  (0, 59664)	0.09770742191644613
  (0, 58514)	0.05250540217562024
  (0, 58403)	0.04014663511243601
  (0, 57662)	0.14368581222288473
  (0, 56871)	0.14107643748845286
  :	:
  (485700, 72656)	0.20663930774149367
  (485700, 71225)	0.26283911340165067
  (485700, 70092)	0.0918765457662843
  (485700, 61605)	0.194740851743595

**Dimensionality Reduction**

In [8]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=5)

principal_components = tSVD.fit_transform(X_train_tfidf)
print(principal_components.shape)

(485701, 5)


__One hot encoding of extra columns__

In [9]:
principal_components_df = pd.DataFrame(principal_components)

df_selected['Issue'] = pd.Categorical(df_selected['Issue'])
df_dummies = pd.get_dummies(df_selected['Issue'])

df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat)

               0         1         2         3         4  \
0       0.187932  0.021532  0.008752 -0.000605 -0.028218   
1       0.123744  0.042873 -0.123670  0.075549 -0.015844   
2       0.177520  0.029086  0.166508 -0.005174 -0.022433   
3       0.196347  0.083273  0.260191  0.082464  0.073285   
4       0.277747  0.005694  0.133969  0.054105 -0.107507   
5       0.194631  0.027033  0.181420  0.017726 -0.040671   
6       0.155707  0.147210 -0.036434 -0.066077 -0.025180   
7       0.187516  0.221281 -0.052110 -0.068515  0.031335   
8       0.070227  0.066873 -0.026217 -0.018804 -0.025971   
9       0.194028 -0.193737  0.004884 -0.078560  0.033764   
10      0.186641  0.129100  0.132839  0.039260  0.065043   
11      0.187083 -0.099723  0.004562 -0.035233  0.006521   
12      0.206061 -0.112380 -0.074631 -0.004835  0.012187   
13      0.245871  0.154648  0.049356 -0.025638  0.023453   
14      0.200793  0.019924 -0.104782  0.045776  0.167353   
15      0.250801  0.159355 -0.169863  0.

**Split into train & test**

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(364275, 166)
(121426, 166)
(364275,)
(121426,)


**Run algorithm**

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

test_predictions = clf.predict(X_test)

**Report**

In [12]:
print(classification_report(test_predictions, Y_test))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3752
                                                 Checking or savings account       0.99      1.00      1.00      4660
                                                               Consumer Loan       0.87      0.80      0.83      2520
                                                                 Credit card       1.00      0.98      0.99      4836
                                                 Credit card or prepaid card       0.95      0.98      0.97      7696
                                                            Credit reporting       1.00      1.00      1.00      7874
Credit reporting, credit repair services, or other personal consumer reports       1.00      0.97      0.98     36326
                                                       